In [ ]:
import pandas as pd
import csv

caminho_arquivo = "ContratosPublicos2024.csv"
separador = '\t' 
PASTA_BD = "BD"
try:
    df_original = pd.read_csv(
        caminho_arquivo,
        sep=separador,
        encoding='latin-1', 
        engine='python',    
        quoting=csv.QUOTE_MINIMAL,
        on_bad_lines='skip' 
    )
    print(f" Ok! {len(df_original)}")
    print(df_original.head())
except Exception as e:
    print(f" ERRO : {e}")
    exit() 


 Ok! 22417


In [24]:
df = pd.read_excel('ContratosPublicos2024.xlsx')
df.head()

,idcontrato,tipoContrato,tipoprocedimento,objectoContrato,adjudicante,adjudicatarios,dataPublicacao,dataCelebracaoContrato,precoContratual,cpv,prazoExecucao,localExecucao,fundamentacao,ProcedimentoCentralizado,DescrAcordoQuadro
0,10424261,Aquisição de serviços,Consulta Prévia,Seguro de Acidentes de Trabalho para os Funcio...,"505111667 - Urbe - Consultores Associados, L.da","506093190 - Penasegur - Mediação de Seguros, Lda",2024-01-01,2024-01-01,3918.75,66512100-3 - Serviços de seguros de acidentes,366,"Portugal, Aveiro, Castelo de Paiva","Artigo 20.º, n.º 1, alínea c) do Código dos Co...",Não,NaN
1,10424271,Aquisição de serviços,Consulta Prévia,Seguro Automóvel - FIAT 500L/1.3MJ LOUNGE,"505111667 - Urbe - Consultores Associados, L.da","506093190 - Penasegur - Mediação de Seguros, Lda",2024-01-02,2024-01-02,467.95,66514110-0 - Serviços de seguro automóvel,366,"Portugal, Aveiro, Castelo de Paiva","Artigo 20.º, n.º 1, alínea c) do Código dos Co...",Não,NaN
2,10424433,Empreitadas de obras públicas,Consulta Prévia,Ligação EN 342 a placa de S. Martinho,506613399 - Município de Góis,510382258 - SOCIASFALTOS ASFALTAGEM DE VIAS DE...,2024-01-02,2024-01-02,27849.00,45233200-1 - Obras diversas de pavimentação,30,"Portugal, Coimbra, Góis","Artigo 19.º, alínea c) do Código dos Contratos...",Não,NaN
3,10424474,Aquisição de serviços,Ajuste Direto Regime Geral,"Serviços de limpeza de bermas, valetas, podas ...",507105141 - Freguesia de Carvalhosa,RGPD - António Fernando Dias de Brito,2024-01-02,2024-01-02,11520.00,90611000-3 - Serviços de limpeza de ruas,365,"Portugal, Porto, Paços de Ferreira","Artigo 20.º, n.º 1, alínea d) do Código dos Co...",Não,NaN
4,10424593,Aquisição de serviços,Consulta Prévia,"Serviços de limpeza de bermas, valetas e manut...",507104676 - Freguesia de Joane,"508725895 - Ambicalendário - Unipessoal, Lda",2024-01-02,2024-01-02,19248.00,90611000-3 - Serviços de limpeza de ruas,365,"Portugal, Braga, Vila Nova de Famalicão","Artigo 20.º, n.º 1, alínea c) do Código dos Co...",Não,NaN


 **Checkpoint** : df(excel) apresenta melhores dados que o df_original(csv)


 Cédula de código abaixo separa certas colunas listadas no ficheiro txt do csv, identificadas como valores compostos
 

In [25]:
def split_code_name(series):
    return series.astype(str).str.split(' - ', n=1, expand=True)

# 1. Tabela ADJUDICANTE
df_adj_split = split_code_name(df['adjudicante'])
T_ADJUDICANTE = df_adj_split.rename(columns={0: 'idAdjudicante', 1: 'NomeAdjudicante'})
T_ADJUDICANTE = T_ADJUDICANTE[['idAdjudicante', 'NomeAdjudicante']].drop_duplicates(subset=['idAdjudicante']).reset_index(drop=True)

# 2. Tabela CPV
df_cpv_split = split_code_name(df['cpv'])
T_CPV = df_cpv_split.rename(columns={0: 'CodigoCPV', 1: 'DescricaoCPV'})
T_CPV = T_CPV[['CodigoCPV', 'DescricaoCPV']].drop_duplicates(subset=['CodigoCPV']).reset_index(drop=True)

# 3. Tabela ADJUDICATARIO e Tabela de LIGAÇÃO (M:N)
df_adjudicatario_split = split_code_name(df['adjudicatarios'])
df_adjudicatario_split = df_adjudicatario_split.rename(columns={0: 'idAdjudicatario', 1: 'NomeAdjudicatario'})
df_adjudicatario_split = df_adjudicatario_split[df_adjudicatario_split['idAdjudicatario'] != np.nan]

T_ADJUDICATARIO = df_adjudicatario_split[['idAdjudicatario', 'NomeAdjudicatario']].drop_duplicates(subset=['idAdjudicatario']).reset_index(drop=True)

T_CONTRATO_ADJUDICATARIO = df[['idcontrato']].copy()
T_CONTRATO_ADJUDICATARIO['idAdjudicatario'] = df_adjudicatario_split['idAdjudicatario']
T_CONTRATO_ADJUDICATARIO = T_CONTRATO_ADJUDICATARIO.drop_duplicates().reset_index(drop=True)

colunas_fato_base = [
    'idcontrato', 'tipoContrato', 'tipoprocedimento', 'objectoContrato', 
    'dataPublicacao', 'dataCelebracaoContrato', 'precoContratual', 
    'prazoExecucao', 'fundamentacao', 'ProcedimentoCentralizado', 
    'DescrAcordoQuadro'
]
T_CONTRATO_FATO = df[colunas_fato_base].copy()

T_CONTRATO_FATO['idAdjudicante_FK'] = df_adj_split[0]
T_CONTRATO_FATO['CodigoCPV_FK'] = df_cpv_split[0]

df_local_split = df['localExecucao'].astype(str).str.split(', ', expand=True)
T_CONTRATO_FATO['Pais'] = df_local_split[0]
T_CONTRATO_FATO['Distrito'] = df_local_split[1]
T_CONTRATO_FATO['Concelho'] = df_local_split[2]

# Imprimir as primeiras linhas das novas tabelas para verificação
print("\nT_ADJUDICANTE:")
print(T_ADJUDICANTE.head())
print("\nT_CPV:")
print(T_CPV.head())
print("\nT_CONTRATO_ADJUDICATARIO:")
print(T_CONTRATO_ADJUDICATARIO.head())
print("\nT_CONTRATO_FATO (Base):")
print(T_CONTRATO_FATO.head())


T_ADJUDICANTE:
  idAdjudicante                                    NomeAdjudicante
0     505111667                Urbe - Consultores Associados, L.da
1     506613399                                  Município de Góis
2     507105141                            Freguesia de Carvalhosa
3     507104676                                 Freguesia de Joane
4     502091835  Associação In Loco de Intervenção, Formação e ...

T_CPV:
    CodigoCPV                      DescricaoCPV
0  66512100-3  Serviços de seguros de acidentes
1  66514110-0      Serviços de seguro automóvel
2  45233200-1    Obras diversas de pavimentação
3  90611000-3       Serviços de limpeza de ruas
4  79800000-2     Serviços de impressão e afins

T_CONTRATO_ADJUDICATARIO:
   idcontrato idAdjudicatario
0    10424261       506093190
1    10424271       506093190
2    10424433       510382258
3    10424474            RGPD
4    10424593       508725895

T_CONTRATO_FATO (Base):
   idcontrato                   tipoContrato          

adjudicante,Atributo composto: Contém o número fiscal e a designação.,Separar em idAdjudicante e NomeAdjudicante.


adjudicatarios,"Atributo composto (número fiscal e designação) e Multivalorado (grupo repetitivo, se houver vários adjudicatários na mesma célula).",1. Separar em idAdjudicatario e NomeAdjudicatario. 2. Criar uma tabela de ligação (N:M) para associar múltiplos adjudicatários a um contrato.


localExecucao,"Atributo composto (País, Distrito, Município) e Multivalorado (""por vezes aparecem valores múltiplos"").","1. Decompor em Pais, Distrito, Concelho. 2. Criar uma tabela de ligação (N:M) entre Contrato e Localização para resolver os múltiplos valores."

Tabela de Ligação: {idcontrato, idAdjudicatario} -> NomeAdjudicatarioO 

NomeAdjudicatario depende apenas do idAdjudicatario (parte da chave composta) e não do idcontrato completo.

cpv,O código cpv (que seria uma FK na tabela principal) determina a descrição do CPV (que seria um atributo não-chave). PK→cpv→Descric\c​a~o.,Mover a descrição do CPV para uma tabela de dimensão separada (T_CPV) onde a descrição depende diretamente do cpv (PK).


localExecucao,"A hierarquia geográfica implica uma dependência transitiva. Se idLocalizacao (PK da tabela de localização) → Distrito, e Distrito → País, então o País depende transitivamente.","Decompor a tabela de Localização, separando os atributos determinantes (ex: criar uma tabela para Distrito/País e outra para Concelho/Distrito)."



tipoContrato / tipoprocedimento,"É provável que tipoprocedimento dependa de tipoContrato. Se ambos forem atributos não-chave na tabela principal, é uma violação da 3FN.",Criar uma tabela separada T_TIPO_CONTRATO que inclua ambas as colunas e seja referenciada por uma chave estrangeira na tabela de contratos.